In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [2]:
reviews_train = []
for line in open('full_train.txt', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('full_test.txt', 'r'):
    
    reviews_test.append(line.strip())

In [3]:
reviews_train[15]

'Like one of the previous commenters said, this had the foundations of a great movie but something happened on the way to delivery. Such a waste because Collette\'s performance was eerie and Williams was believable. I just kept waiting for it to get better. I don\'t think it was bad editing or needed another director, it could have just been the film. It came across as a Canadian movie, something like the first few seasons of X-Files. Not cheap, just hokey. Also, it needed a little more suspense. Something that makes you jump off your seat. The movie reached that moment then faded away; kind of like a false climax. I can see how being too suspenseful would have taken away from the "reality" of the story but I thought that part was reached when Gabriel was in the hospital looking for the boy. This movie needs to have a Director\'s cut that tries to fix these problems.'

In [4]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [5]:
reviews_train_clean[15]

'like one of the previous commenters said this had the foundations of a great movie but something happened on the way to delivery such a waste because collettes performance was eerie and williams was believable i just kept waiting for it to get better i dont think it was bad editing or needed another director it could have just been the film it came across as a canadian movie something like the first few seasons of x files not cheap just hokey also it needed a little more suspense something that makes you jump off your seat the movie reached that moment then faded away kind of like a false climax i can see how being too suspenseful would have taken away from the reality of the story but i thought that part was reached when gabriel was in the hospital looking for the boy this movie needs to have a directors cut that tries to fix these problems'

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.87456
Accuracy for C=0.05: 0.88304


/Users/bruce/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy for C=0.25: 0.87952


/Users/bruce/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy for C=0.5: 0.87712
Accuracy for C=1: 0.87552


/Users/bruce/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [8]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))

Final Accuracy: 0.88144


In [9]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

('excellent', 0.9283544333476483)
('perfect', 0.7944277845461106)
('great', 0.6745552731960056)
('amazing', 0.6164834577729078)
('superb', 0.605591966214955)
('worst', -1.3679897734293813)
('waste', -1.1688808917878097)
('awful', -1.0273337546642216)
('poorly', -0.8748022404777991)
('boring', -0.8591221218897568)
